# sentenceBERT
* https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2

In [1]:
%%capture
!pip install transformers
!pip install fugashi
!pip install ipadic

In [5]:
from transformers import BertJapaneseTokenizer, BertModel
import torch


class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)


In [6]:
MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"  # <- v2です。
model = SentenceBertJapanese(MODEL_NAME)

sentences = ["量子コンピュータを使うデータサイエンティスト"]
sentence_embeddings = model.encode(sentences, batch_size=8)

print("Sentence embeddings:", sentence_embeddings)

Sentence embeddings: tensor([[-8.4269e-01,  8.6399e-01,  7.0584e-01, -1.8740e-02,  9.8217e-01,
         -8.3913e-01,  1.5679e+00, -5.5971e-01,  8.4783e-03,  1.3139e+00,
          4.2518e-01,  6.0655e-01, -9.0300e-01, -7.1108e-01, -1.4093e-01,
         -1.6973e-02,  1.7206e-01,  7.2887e-01,  1.2516e-01, -1.5166e-01,
          2.3493e-01,  8.2472e-02,  4.3417e-01, -5.9426e-01, -7.9133e-01,
         -2.2210e-01, -8.2877e-01, -3.1285e-01,  4.7588e-01,  8.1095e-01,
          2.9929e-01, -8.2731e-01,  5.8782e-01,  9.9275e-01, -4.7253e-01,
          2.8159e-01, -2.5384e-02,  9.9497e-01,  5.4013e-01, -1.3688e+00,
         -8.8183e-01,  1.7397e+00, -1.3928e+00,  4.1439e-01,  7.4225e-01,
          7.2051e-01,  4.0400e-01,  2.8564e-01,  2.7980e-01, -1.1960e+00,
          8.6408e-01,  4.5193e-01,  2.0175e-01, -1.0968e-01, -6.5506e-01,
          4.4915e-01,  2.6203e-01,  2.0201e-01,  1.1133e+00,  1.2912e+00,
         -4.8711e-01, -9.0625e-01,  4.5924e-01, -2.8673e-01, -1.2116e-02,
         -1.2662e